In [1]:
import branca
import folium
import pandas as pd
import geopandas as gpd
from folium import plugins
from branca import colormap as cm
from ipywidgets import interact, fixed

Unable to open EPSG support file gcs.csv.  Try setting the GDAL_DATA environment variable to point to the directory containing EPSG csv files.


In [2]:
karelia = gpd.read_file('karelia.geojson', encoding='utf8')
columns = ['name', 'population', 'geometry']
karelia = karelia[columns].dropna()
karelia.head()

,name,population,geometry
0,Калевальский район,7525,"POLYGON ((29.9307181 65.6787257, 29.900922 65...."
1,Лоухский район,12872,"POLYGON ((29.9813464 66.02932730000001, 29.984..."
2,Медвежьегорский район,29874,"POLYGON ((35.6805005 62.1908169, 35.7515598 62..."
3,Муезерский район,11214,"POLYGON ((31.4390222 63.0362137, 31.8226383 63..."
4,Лахденпохский район,13712,"POLYGON ((29.7514237 61.1723842, 29.7547277 61..."


In [3]:
to_swap = {"Петрозаводский городской округ": "Петрозаводск", "Костомукшский городской округ": "Костомукша"}
karelia.name = karelia.name.apply(lambda x: str(x).rstrip("район").rstrip(" "))
karelia.name = karelia.name.apply(lambda x: to_swap[x] if x in to_swap else x)
karelia.head()

,name,population,geometry
0,Калевальский,7525,"POLYGON ((29.9307181 65.6787257, 29.900922 65...."
1,Лоухский,12872,"POLYGON ((29.9813464 66.02932730000001, 29.984..."
2,Медвежьегорский,29874,"POLYGON ((35.6805005 62.1908169, 35.7515598 62..."
3,Муезерский,11214,"POLYGON ((31.4390222 63.0362137, 31.8226383 63..."
4,Лахденпохский,13712,"POLYGON ((29.7514237 61.1723842, 29.7547277 61..."


In [4]:
fire_df = pd.read_csv('firemen_karelia_restruct.csv')
fire_df.head()

,Район,Год,Количество пожаров,Материальный ущерб (тыс. руб.),Количество пострадавших,Количество ДПФ,Количество добровольцев,Количество выездов добровольцев на пожары,Загорания,Количество обученных добровольцев,Субъект РФ
0,Беломорский,2015,42,1009,7,32,181,6,0,128,Республика Карелия
1,Беломорский,2016,38,1150,5,31,183,9,6,135,Республика Карелия
2,Беломорский,2017,34,262,6,34,220,4,3,173,Республика Карелия
3,Беломорский,2018,32,240,4,32,214,5,0,185,Республика Карелия
4,Калевальский,2015,25,1873,2,14,101,12,0,28,Республика Карелия


In [5]:
data = pd.merge(karelia, fire_df, how="left", left_on="name", right_on="Район").drop(columns=["Район"])

In [6]:
data.head()

,name,population,geometry,Год,Количество пожаров,Материальный ущерб (тыс. руб.),Количество пострадавших,Количество ДПФ,Количество добровольцев,Количество выездов добровольцев на пожары,Загорания,Количество обученных добровольцев,Субъект РФ
0,Калевальский,7525,"POLYGON ((29.9307181 65.6787257, 29.900922 65....",2015.0,25.0,1873.0,2.0,14.0,101.0,12.0,0.0,28.0,Республика Карелия
1,Калевальский,7525,"POLYGON ((29.9307181 65.6787257, 29.900922 65....",2016.0,25.0,407.0,10.0,14.0,106.0,6.0,0.0,28.0,Республика Карелия
2,Калевальский,7525,"POLYGON ((29.9307181 65.6787257, 29.900922 65....",2017.0,20.0,249.0,3.0,15.0,111.0,5.0,0.0,62.0,Республика Карелия
3,Калевальский,7525,"POLYGON ((29.9307181 65.6787257, 29.900922 65....",2018.0,14.0,0.0,0.0,15.0,117.0,0.0,3.0,83.0,Республика Карелия
4,Лоухский,12872,"POLYGON ((29.9813464 66.02932730000001, 29.984...",2015.0,31.0,171.0,6.0,22.0,148.0,3.0,0.0,77.0,Республика Карелия


In [7]:
data.rename({"Количество пожаров": "fire_count", "Количество ДПФ": "dpf_count",
             "Год": "year"}, axis='columns', inplace=True)

In [8]:
data.year = data.year.astype("int", errors="ignore")

In [9]:
colormap = cm.linear.YlOrRd_04.scale(
    data.fire_count.quantile(0.05),
    data.fire_count.quantile(0.95)
)

In [10]:
def style_fire(feature):
    val = data.fire_count.iloc[int(feature['id'])]
    out = {
        'weight': 0.8,
        'color': 'black',
        'opacity': 0.6
    }
    if str(val) == 'nan':
        out['fillOpacity'] = 0
    else:
        out['fillColor'] = colormap(val),
        out['fillOpacity'] = 0.6
    return out

In [11]:
def highlight_fire(feature):
    val = data.fire_count.iloc[int(feature['id'])]
    if str(val) == 'nan':
        return {'fillOpacity': 0}
    return {
        'fillOpacity': 0.8,
        'weight':1.4,
    }

In [12]:
colormap_dpf = cm.linear.YlGn_06.scale(
    data.dpf_count.quantile(0.05),
    data.dpf_count.quantile(0.95)
)

In [13]:
def style_dpf(feature):
    val = data.dpf_count.iloc[int(feature['id'])]
    out = {
        'weight': 0.8,
        'color': 'black',
        'opacity': 0.6
    }
    if str(val) == 'nan':
        out['fillOpacity'] = 0
    else:
        out['fillColor'] = colormap_dpf(val),
        out['fillOpacity'] = 0.6
    return out

In [14]:
def highlight_dpf(feature):
    val = data.dpf_count.iloc[int(feature['id'])]
    if str(val) == 'nan':
        return {'fillOpacity': 0}
    return {
        'fillOpacity': 0.8,
        'weight':1.4,
    }

In [15]:
@interact(location=fixed([64, 33]), zoom_start=fixed(7), Year=(2015, 2018))
def render_map(location, zoom_start, Year):
    dmap = plugins.DualMap(location=location, zoom_start=zoom_start)
    
    tooltip = folium.GeoJsonTooltip(
        fields=["name", "population", "fire_count"],
        aliases=["Район", "Население", "Пожаров"]
    )

    folium.GeoJson(
        data[data.year == Year],
        style_function=style_fire,
        highlight_function=highlight_fire,
        tooltip=tooltip
    ).add_to(dmap.m1)

    colormap.caption = "Количество пожаров"
    colormap.add_to(dmap.m1)
    
    tooltip_dpf = folium.GeoJsonTooltip(
        fields=["name", "population", "dpf_count"],
        aliases=["Район", "Население", "Добровольных пожарных\nформиров."]
    )

    folium.GeoJson(
        data[data.year == Year],
        style_function=style_dpf,
        highlight_function=highlight_dpf,
        tooltip=tooltip_dpf
    ).add_to(dmap.m2)

    colormap_dpf.caption = "Количество ДПФ"
    colormap_dpf.add_to(dmap.m1)
    
    return dmap

interactive(children=(IntSlider(value=2016, description='Year', max=2018, min=2015), Output()), _dom_classes=(…